In [88]:
class GameObject:
    """Base class for all game objects (items, characters, locations)"""
    def __init__(self, name, description=""):
        self.name = name
        self.description = description
        self.properties = {}

    def get_property(self, key, default=None):
        return self.properties.get(key, default)

    def set_property(self, key, value):
        self.properties[key] = value


class Item(GameObject):
    """Represents an item in the game"""
    def __init__(self, name, description=""):
        super().__init__(name, description)


class Character(GameObject):
    """Represents a character (player or NPC)"""
    def __init__(self, name, location=None):
        super().__init__(name)
        self.location = location
        self.inventory = {}

    def add_to_inventory(self, item):
        self.inventory[item.name] = item

    def remove_from_inventory(self, item):
        if item.name in self.inventory:
            del self.inventory[item.name]

    def is_in_inventory(self, item):
        return item.name in self.inventory


class Location(GameObject):
    """Represents a location in the game"""
    def __init__(self, name, description=""):
        super().__init__(name, description)
        self.characters = {}
        self.items = {}

    def here(self, character):
        """Check if a character is in this location"""
        return character.location == self

    def add_character(self, character):
        self.characters[character.name] = character
        character.location = self

    def remove_character(self, character):
        if character.name in self.characters:
            del self.characters[character.name]

    def add_item(self, item):
        self.items[item.name] = item

    def remove_item(self, item):
        if item.name in self.items:
            del self.items[item.name]


class Parser:
    """Simple parser for matching game objects"""
    def __init__(self, game):
        self.game = game

    def get_character(self, text):
        """Find a character by name in text"""
        # If already a Character object, return it
        if isinstance(text, Character):
            return text

        text = text.lower().strip()
        for char in self.game.characters.values():
            if char.name.lower() in text:
                return char
        return None

    def match_item(self, text, inventory_dict):
        """Find an item by name in inventory"""
        # If already an Item object, return it
        if isinstance(text, Item):
            return text

        text = text.lower().strip()
        for item_name, item in inventory_dict.items():
            if item_name.lower() in text:
                return item
        return None

    def ok(self, message):
        """Print success message"""
        print(f"✓ {message}")

    def fail(self, message):
        """Print failure message"""
        print(f"✗ {message}")


class Game:
    """Main game class"""
    def __init__(self):
        self.player = None
        self.characters = {}
        self.locations = {}
        self.items = {}
        self.state = {}
        self.parser = Parser(self)

    def add_character(self, character):
        self.characters[character.name] = character

    def add_location(self, location):
        self.locations[location.name] = location

    def add_item(self, item):
        self.items[item.name] = item

    def get_item(self, item_name):
        return self.items.get(item_name)

In [89]:
class Action:
    """Base Action class"""
    ACTION_NAME = ""
    ACTION_DESCRIPTION = ""
    ACTION_ALIASES = []

    def __init__(self, game):
        self.game = game
        self.parser = game.parser

    def was_matched(self, obj, error_message=None):
        """Check if object was successfully matched"""
        if obj is None:
            if error_message:
                self.parser.fail(error_message)
            return False
        return True

    def check_preconditions(self) -> bool:
        """Override in subclasses"""
        return True

    def apply_effects(self):
        """Override in subclasses"""
        pass

    def execute(self):
        """Execute the action if preconditions are met"""
        if self.check_preconditions():
            self.apply_effects()
            return True
        return False

In [90]:
class Attack(Action):
    ACTION_NAME = "attack"
    ACTION_DESCRIPTION = "Attack someone with a weapon"
    ACTION_ALIASES = ["hit", "stab", "shoot", "strike", "slash"]

    def __init__(self, game, victim, weapon):
        super().__init__(game)
        self.attacker = game.player
        self.victim = victim
        self.weapon = weapon

    def check_preconditions(self) -> bool:
        if not self.was_matched(self.victim, "The target couldn't be found."):
            return False
        if not self.attacker.location.here(self.victim):
            self.parser.fail("The target is not here.")
            return False
        if not self.was_matched(self.weapon, "You need a weapon to attack."):
            return False
        if not self.attacker.is_in_inventory(self.weapon):
            self.parser.fail(f"You don't have the {self.weapon.name}.")
            return False
        if not self.weapon.get_property("is_weapon"):
            self.parser.fail(f"{self.weapon.name} is not a weapon.")
            return False
        if self.victim.get_property("is_dead"):
            self.parser.fail(f"{self.victim.name} is already dead.")
            return False
        return True

    def apply_effects(self):
        weapon_damage = self.weapon.get_property("damage", 10)
        victim_health = self.victim.get_property("health", 100)

        new_health = max(0, victim_health - weapon_damage)
        self.victim.set_property("health", new_health)

        self.parser.ok(f"You attack {self.victim.name} with the {self.weapon.name} for {weapon_damage} damage.")

        if self.victim.get_property("is_invulnerable"):
            self.parser.ok(f"The attack has no effect on {self.victim.name}!")
            return

        if new_health <= 0:
            self.victim.set_property("is_dead", True)
            self.parser.ok(f"{self.victim.name} has been slain.")

            blood_echoes = self.victim.get_property("blood_echoes_drop", 0)
            if blood_echoes > 0:
                player_echoes = self.game.player.get_property("blood_echoes", 0)
                self.game.player.set_property("blood_echoes", player_echoes + blood_echoes)
                self.parser.ok(f"Gained {blood_echoes} Blood Echoes.")

            if self.victim.get_property("is_boss"):
                insight = self.game.player.get_property("insight", 0)
                self.game.player.set_property("insight", insight + 1)
                self.parser.ok("Insight gained.")

            for item_name in list(self.victim.inventory.keys()):
                item = self.victim.inventory[item_name]
                self.victim.remove_from_inventory(item)
                self.victim.location.add_item(item)
                self.parser.ok(f"{self.victim.name} dropped {item_name}.")


In [91]:
class Talk(Action):
    ACTION_NAME = "talk"
    ACTION_DESCRIPTION = "Talk to an NPC"
    ACTION_ALIASES = ["speak", "chat", "communicate"]

    def __init__(self, game, npc):
        super().__init__(game)
        self.speaker = game.player
        self.npc = npc

    def check_preconditions(self) -> bool:
        if not self.was_matched(self.npc, "There's no one here by that name."):
            return False
        if not self.speaker.location.here(self.npc):
            self.parser.fail(f"{self.npc.name} is not here.")
            return False
        if self.npc.get_property("is_dead"):
            self.parser.fail(f"{self.npc.name} is dead and cannot speak.")
            return False
        if self.npc.get_property("is_hostile"):
            self.parser.fail(f"{self.npc.name} is hostile and won't talk.")
            return False
        return True

    def apply_effects(self):
        player_insight = self.game.player.get_property("insight", 0)
        dialogue_state = self.npc.get_property("dialogue_state", 0)

        dialogues = self.npc.get_property("dialogues", {})
        dialogue_key = f"state_{dialogue_state}_insight_{player_insight}"

        if dialogue_key in dialogues:
            dialogue = dialogues[dialogue_key]
        elif f"state_{dialogue_state}" in dialogues:
            dialogue = dialogues[f"state_{dialogue_state}"]
        else:
            dialogue = dialogues.get("default", f"{self.npc.name} has nothing to say.")

        self.parser.ok(f'{self.npc.name}: "{dialogue}"')
        self.npc.set_property("dialogue_state", dialogue_state + 1)


In [92]:
class Use(Action):
    ACTION_NAME = "use"
    ACTION_DESCRIPTION = "Use an item from inventory"
    ACTION_ALIASES = ["consume", "activate"]

    def __init__(self, game, item):
        super().__init__(game)
        self.user = game.player
        self.item = item

    def check_preconditions(self) -> bool:
        if not self.was_matched(self.item, "You don't have that item."):
            return False
        if not self.user.is_in_inventory(self.item):
            self.parser.fail(f"You don't have {self.item.name}.")
            return False
        if not self.item.get_property("is_usable"):
            self.parser.fail(f"{self.item.name} cannot be used.")
            return False
        return True

    def apply_effects(self):
        item_type = self.item.get_property("item_type", "generic")

        if item_type == "healing":
            heal_amount = self.item.get_property("heal_amount", 50)
            current_health = self.user.get_property("health", 100)
            max_health = self.user.get_property("max_health", 100)
            new_health = min(max_health, current_health + heal_amount)
            self.user.set_property("health", new_health)
            self.parser.ok(f"You used {self.item.name}. Restored {heal_amount} HP.")

        elif item_type == "key_item":
            self.parser.ok(f"You used {self.item.name}.")
            self.game.state[f"used_{self.item.name.lower().replace(' ', '_')}"] = True

        else:
            self.parser.ok(f"You used {self.item.name}.")

        if self.item.get_property("is_consumable"):
            self.user.remove_from_inventory(self.item)

In [93]:
class Give(Action):
    ACTION_NAME = "give"
    ACTION_DESCRIPTION = "Give an item to an NPC"
    ACTION_ALIASES = ["trade", "offer"]

    def __init__(self, game, receiver, item):
        super().__init__(game)
        self.giver = game.player
        self.receiver = receiver
        self.item = item

    def check_preconditions(self) -> bool:
        if not self.was_matched(self.receiver, "There's no one here by that name."):
            return False
        if not self.giver.location.here(self.receiver):
            self.parser.fail(f"{self.receiver.name} is not here.")
            return False
        if not self.was_matched(self.item, "You don't have that item."):
            return False
        if not self.giver.is_in_inventory(self.item):
            self.parser.fail(f"You don't have {self.item.name}.")
            return False
        if self.receiver.get_property("is_dead"):
            self.parser.fail(f"{self.receiver.name} is dead.")
            return False
        if self.receiver.get_property("is_hostile"):
            self.parser.fail(f"{self.receiver.name} won't accept anything from you.")
            return False
        return True

    def apply_effects(self):
        wanted_items = self.receiver.get_property("wants_items", [])

        self.giver.remove_from_inventory(self.item)
        self.receiver.add_to_inventory(self.item)

        self.parser.ok(f"You gave {self.item.name} to {self.receiver.name}.")

        if self.item.name in wanted_items:
            reward_item = self.receiver.get_property("trade_reward")
            if reward_item:
                self.giver.add_to_inventory(reward_item)
                self.parser.ok(f"{self.receiver.name} gave you {reward_item.name} in return.")

            self.game.state[f"gave_{self.item.name}_to_{self.receiver.name}"] = True


In [94]:
class Upgrade_Weapon(Action):
    ACTION_NAME = "upgrade_weapon"
    ACTION_DESCRIPTION = "Upgrade a weapon using Blood Echoes"
    ACTION_ALIASES = ["upgrade", "enhance", "fortify"]

    def __init__(self, game, weapon):
        super().__init__(game)
        self.player = game.player
        self.weapon = weapon

    def check_preconditions(self) -> bool:
        if not self.was_matched(self.weapon, "You don't have that weapon."):
            return False
        if not self.player.is_in_inventory(self.weapon):
            self.parser.fail(f"You don't have {self.weapon.name}.")
            return False
        if not self.weapon.get_property("is_weapon"):
            self.parser.fail(f"{self.weapon.name} is not a weapon.")
            return False

        current_level = self.weapon.get_property("upgrade_level", 0)
        max_level = self.weapon.get_property("max_upgrade_level", 10)
        if current_level >= max_level:
            self.parser.fail(f"{self.weapon.name} is already at maximum level.")
            return False

        upgrade_cost = self.weapon.get_property("upgrade_cost", 100) * (current_level + 1)
        player_echoes = self.player.get_property("blood_echoes", 0)
        if player_echoes < upgrade_cost:
            self.parser.fail(f"You need {upgrade_cost} Blood Echoes. You have {player_echoes}.")
            return False

        at_workshop = self.player.location.get_property("is_workshop", False)
        if not at_workshop:
            self.parser.fail("You must be at a workshop to upgrade weapons.")
            return False

        return True

    def apply_effects(self):
        current_level = self.weapon.get_property("upgrade_level", 0)
        upgrade_cost = self.weapon.get_property("upgrade_cost", 100) * (current_level + 1)

        player_echoes = self.player.get_property("blood_echoes", 0)
        self.player.set_property("blood_echoes", player_echoes - upgrade_cost)

        new_level = current_level + 1
        self.weapon.set_property("upgrade_level", new_level)

        current_damage = self.weapon.get_property("damage", 10)

        new_damage = int(current_damage * 1.2)
        self.weapon.set_property("damage", new_damage)

        self.parser.ok(f"Upgraded {self.weapon.name} from {current_damage} to +{new_level}. Damage: {new_damage}")

In [95]:
class Heal(Action):
    ACTION_NAME = "heal"
    ACTION_DESCRIPTION = "Heal yourself"
    ACTION_ALIASES = ["restore", "cure"]

    def __init__(self, game):
        super().__init__(game)
        self.healer = game.player
        self.target = game.player
        self.item = None
        for item in self.healer.inventory.values():
            if item.get_property("item_type") == "healing":
                self.item = item
                break

    def check_preconditions(self) -> bool:
        if not self.item:
            self.parser.fail("You don't have any healing items.")
            return False
        if self.target.get_property("is_dead"):
            self.parser.fail("You are dead and cannot be healed.")
            return False
        return True

    def apply_effects(self):
        heal_amount = self.item.get_property("heal_amount", 50)
        current_health = self.target.get_property("health", 100)
        max_health = self.target.get_property("max_health", 100)
        new_health = min(max_health, current_health + heal_amount)

        self.target.set_property("health", new_health)
        self.parser.ok(f"Healed for {heal_amount} HP. Current health: {new_health}/{max_health}")

        if self.item.get_property("is_consumable"):
            self.healer.remove_from_inventory(self.item)

In [96]:
class Travel(Action):
    ACTION_NAME = "travel"
    ACTION_DESCRIPTION = "Travel between locations"
    ACTION_ALIASES = ["go", "move", "teleport", "warp"]

    def __init__(self, game, destination):
        super().__init__(game)
        self.player = game.player
        self.from_location = self.player.location

        if isinstance(destination, Location):
            self.to_location = destination
        else:
            self.to_location = None
            for loc_name, location in game.locations.items():
                if loc_name.lower() in destination.lower():
                    self.to_location = location
                    break

    def check_preconditions(self) -> bool:
        if not self.to_location:
            self.parser.fail("That location doesn't exist.")
            return False

        adjacent_locations = self.from_location.get_property("adjacent_locations", [])
        if self.to_location.name in adjacent_locations:
            # Can walk there
            return True

        at_lamp = self.from_location.get_property("has_lamp", False)
        dest_has_lamp = self.to_location.get_property("has_lamp", False)
        dest_lamp_lit = self.to_location.get_property("lamp_lit", False)

        if at_lamp and dest_has_lamp and dest_lamp_lit:
            # Can teleport
            return True
        elif at_lamp and dest_has_lamp and not dest_lamp_lit:
            self.parser.fail("You haven't lit the lamp at that location yet.")
            return False
        else:
            self.parser.fail("You can't reach that location from here")
            return False

    def apply_effects(self):
        self.from_location.remove_character(self.player)
        self.to_location.add_character(self.player)

        at_lamp = self.from_location.get_property("has_lamp", False)
        if at_lamp:
            self.parser.ok(f"Teleported to {self.to_location.name}.")
        else:
            self.parser.ok(f"Traveled to {self.to_location.name}.")

        insight = self.player.get_property("insight", 0)
        descriptions = self.to_location.get_property("descriptions", {})

        if f"insight_{insight}" in descriptions:
            description = descriptions[f"insight_{insight}"]
        else:
            description = descriptions.get("default", self.to_location.description)

        print(f"\n{description}\n")

In [97]:
class Interact(Action):
    ACTION_NAME = "interact"
    ACTION_DESCRIPTION = "Interact with objects in the environment"
    ACTION_ALIASES = ["examine", "touch", "activate"]

    def __init__(self, game, target):
        super().__init__(game)
        self.player = game.player

        if isinstance(target, Item):
            self.target = target
        else:
            self.target = self.parser.match_item(target, self.player.location.items)

    def check_preconditions(self) -> bool:
        if not self.was_matched(self.target, "There's nothing like that here."):
            return False
        return True

    def apply_effects(self):
        if self.target.get_property("is_lamp"):
            if self.target.get_property("is_lit"):
                self.parser.ok("This lamp is already lit.")
            else:
                self.target.set_property("is_lit", True)
                self.player.location.set_property("lamp_lit", True)
                self.parser.ok("You lit the lamp. You can now travel here from other lamps.")

                insight = self.player.get_property("insight", 0)
                self.player.set_property("insight", insight + 1)

        elif self.target.get_property("is_interactive"):
            interaction_text = self.target.get_property("interaction_text", "Nothing happens.")
            self.parser.ok(interaction_text)

            event = self.target.get_property("triggers_event")
            if event:
                self.game.state[event] = True

        else:
            self.parser.ok(f"You interact with {self.target.name}, but nothing happens.")

In [98]:
class Get(Action):
    ACTION_NAME = "get"
    ACTION_DESCRIPTION = "Pick up an item"
    ACTION_ALIASES = ["take", "grab", "pickup", "collect"]

    def __init__(self, game, item):
        super().__init__(game)
        self.player = game.player

        if isinstance(item, Item):
            self.item = item
        else:
            # Parse from string
            self.item = self.parser.match_item(item, self.player.location.items)

    def check_preconditions(self) -> bool:
        if not self.was_matched(self.item, "There's no such item here."):
            return False
        if not self.item.get_property("is_takeable", True):
            self.parser.fail(f"You can't take {self.item.name}.")
            return False
        return True

    def apply_effects(self):
        self.player.location.remove_item(self.item)
        self.player.add_to_inventory(self.item)
        self.parser.ok(f"You picked up {self.item.name}.")

        description = self.item.get_property("pickup_description")
        if description:
            print(f"  {description}")

In [99]:
class Inspect(Action):
    ACTION_NAME = "inspect"
    ACTION_DESCRIPTION = "Examine an item or character for details"
    ACTION_ALIASES = ["examine", "look", "check", "read"]

    def __init__(self, game, target):
        super().__init__(game)
        self.player = game.player

        if isinstance(target, (Item, Character)):
            self.target = target
        else:
            self.target = (
                self.parser.match_item(target, self.player.inventory) or
                self.parser.match_item(target, self.player.location.items) or
                self.parser.get_character(target)
            )

    def check_preconditions(self) -> bool:
        if not self.was_matched(self.target, "You don't see that here."):
            return False
        return True

    def apply_effects(self):
        base_description = self.target.get_property("description", self.target.description)
        self.parser.ok(base_description)

        insight = self.player.get_property("insight", 0)
        insight_desc = self.target.get_property(f"insight_{insight}_description")
        if insight_desc:
            print(f"\n[With your insight, you notice: {insight_desc}]")

        lore = self.target.get_property("lore")
        if lore:
            print(f"\nLore: {lore}")


In [100]:
class Open(Action):
    ACTION_NAME = "open"
    ACTION_DESCRIPTION = "Open doors, gates, or containers"
    ACTION_ALIASES = ["unlock"]

    def __init__(self, game, target, key=None):
        super().__init__(game)
        self.player = game.player

        if isinstance(target, Item):
            self.target = target
        else:
            self.target = self.parser.match_item(target, self.player.location.items)

        if key and isinstance(key, Item):
            self.key = key
        elif key:
            self.key = self.parser.match_item(key, self.player.inventory)
        else:
            self.key = None

    def check_preconditions(self) -> bool:
        if not self.was_matched(self.target, "There's nothing to open here."):
            return False
        if not self.target.get_property("is_openable"):
            self.parser.fail(f"{self.target.name} cannot be opened.")
            return False
        if self.target.get_property("is_open"):
            self.parser.fail(f"{self.target.name} is already open.")
            return False

        is_locked = self.target.get_property("is_locked", False)
        if is_locked and not self.key:
            required_key = self.target.get_property("requires_key", "a key")
            self.parser.fail(f"{self.target.name} is locked. You need {required_key}.")
            return False

        return True

    def apply_effects(self):
        self.target.set_property("is_open", True)

        if self.target.get_property("is_locked"):
            self.target.set_property("is_locked", False)
            self.parser.ok(f"You unlocked {self.target.name} with {self.key.name}.")

        self.parser.ok(f"You opened {self.target.name}.")

        contents = self.target.get_property("contains", [])
        if contents:
            self.parser.ok(f"Inside you find: {', '.join(contents)}")
            for item_name in contents:
                item = self.game.get_item(item_name)
                if item:
                    self.player.location.add_item(item)

        opens_path = self.target.get_property("opens_path_to")
        if opens_path:
            adjacent = self.player.location.get_property("adjacent_locations", [])
            adjacent.append(opens_path)
            self.player.location.set_property("adjacent_locations", adjacent)
            self.parser.ok(f"A path to {opens_path} is now open.")

In [101]:
class Drop(Action):
    ACTION_NAME = "drop"
    ACTION_DESCRIPTION = "Drop an item from inventory"
    ACTION_ALIASES = ["discard", "leave"]

    def __init__(self, game, item):
        super().__init__(game)
        self.player = game.player

        if isinstance(item, Item):
            self.item = item
        else:
            self.item = self.parser.match_item(item, self.player.inventory)

    def check_preconditions(self) -> bool:
        if not self.was_matched(self.item, "You don't have that item."):
            return False
        if not self.player.is_in_inventory(self.item):
            self.parser.fail(f"You don't have {self.item.name}.")
            return False
        return True

    def apply_effects(self):
        self.player.remove_from_inventory(self.item)
        self.player.location.add_item(self.item)
        self.parser.ok(f"You dropped {self.item.name}.")


In [102]:
def create_demo_game():
    """Create a demo game to test the actions"""

    game = Game()

    #  locations
    hunters_dream = Location("Hunter's Dream", "A safe haven bathed in pale moonlight.")
    hunters_dream.set_property("is_workshop", True)
    hunters_dream.set_property("has_lamp", True)
    hunters_dream.set_property("lamp_lit", True)
    hunters_dream.set_property("adjacent_locations", ["Central Yharnam"])

    central_yharnam = Location("Central Yharnam", "A decrepit street filled with the stench of beasts.")
    central_yharnam.set_property("adjacent_locations", ["Hunter's Dream"])
    central_yharnam.set_property("has_lamp", True)
    central_yharnam.set_property("lamp_lit", False)
    central_yharnam.set_property("descriptions", {
        "default": "A decrepit street filled with the stench of beasts.",
        "insight_2": "The street is shrouded in an otherworldly mist. You sense eyes watching from the shadows."
    })

    game.add_location(hunters_dream)
    game.add_location(central_yharnam)

    #  player
    player = Character("Hunter")
    hunters_dream.add_character(player)
    player.set_property("health", 100)
    player.set_property("max_health", 100)
    player.set_property("blood_echoes", 500)
    player.set_property("insight", 0)
    game.player = player
    game.add_character(player)

    #  weapons
    saw_cleaver = Item("Saw Cleaver")
    saw_cleaver.set_property("is_weapon", True)
    saw_cleaver.set_property("is_usable", False)
    saw_cleaver.set_property("damage", 25)
    saw_cleaver.set_property("upgrade_level", 0)
    saw_cleaver.set_property("max_upgrade_level", 10)
    saw_cleaver.set_property("upgrade_cost", 100)
    saw_cleaver.set_property("description", "A crude but effective weapon of the hunt.")
    player.add_to_inventory(saw_cleaver)
    game.add_item(saw_cleaver)

    pistol = Item("Hunter Pistol")
    pistol.set_property("is_weapon", True)
    pistol.set_property("is_usable", False)
    pistol.set_property("damage", 15)
    pistol.set_property("description", "A firearm used to parry and stagger beasts.")
    player.add_to_inventory(pistol)
    game.add_item(pistol)

    #  healing item
    blood_vial = Item("Blood Vial")
    blood_vial.set_property("is_usable", True)
    blood_vial.set_property("is_consumable", True)
    blood_vial.set_property("item_type", "healing")
    blood_vial.set_property("heal_amount", 40)
    blood_vial.set_property("description", "A vial of restorative blood.")
    player.add_to_inventory(blood_vial)
    game.add_item(blood_vial)

    #  NPC
    gascoigne = Character("Father Gascoigne")
    central_yharnam.add_character(gascoigne)
    gascoigne.set_property("health", 150)
    gascoigne.set_property("is_boss", True)
    gascoigne.set_property("blood_echoes_drop", 300)
    gascoigne.set_property("is_hostile", False)
    gascoigne.set_property("dialogues", {
        "state_0": "Beasts all over the shop... You'll be one of them, sooner or later.",
        "state_1": "Too proud to show your true face, eh? But a sporting hunt it was."
    })
    game.add_character(gascoigne)

    #  lamp item
    lamp = Item("Lantern")
    lamp.set_property("is_lamp", True)
    lamp.set_property("is_lit", False)
    lamp.set_property("is_takeable", False)
    lamp.set_property("description", "A cold, dark lantern.")
    central_yharnam.add_item(lamp)
    game.add_item(lamp)

    return game



In [103]:
if __name__ == "__main__":
    print("="*60)
    print("BLOODBORNE TEXT ADVENTURE - ACTION SYSTEM DEMO")
    print("="*60)
    print()

    game = create_demo_game()

    print(f"Location: {game.player.location.name}")
    print(f"Health: {game.player.get_property('health')}/{game.player.get_property('max_health')}")
    print(f"Blood Echoes: {game.player.get_property('blood_echoes')}")
    print(f"Insight: {game.player.get_property('insight')}")
    print(f"Inventory: {', '.join(game.player.inventory.keys())}")
    print()

    print("\n--- TEST 1: UPGRADE WEAPON ---")
    saw_cleaver = game.player.inventory["Saw Cleaver"]
    action = Upgrade_Weapon(game, saw_cleaver)
    action.execute()

    print("\n--- TEST 2: TRAVEL ---")
    central_yharnam = game.locations["Central Yharnam"]
    action = Travel(game, central_yharnam)
    action.execute()

    print("\n--- TEST 2B: TRAVEL BY WALKING ---")
    action = Travel(game, central_yharnam)
    if action.check_preconditions():
        print("(Checking adjacent locations...)")
        print(f"Current location: {game.player.location.name}")
        print(f"Adjacent to Hunter's Dream: {hunters_dream.get_property('adjacent_locations')}")
    action.execute()

    print(f"Player is now at: {game.player.location.name}")

    print("\n--- TEST 3: INTERACT WITH LAMP ---")
    lamp = game.player.location.items["Lantern"]
    action = Interact(game, lamp)
    action.execute()

    print("\n--- TEST 4: TALK TO NPC ---")
    gascoigne = game.characters["Father Gascoigne"]
    action = Talk(game, gascoigne)
    action.execute()

    print("\n--- TEST 5: ATTACK ---")
    pistol = game.player.inventory["Hunter Pistol"]
    action = Attack(game, gascoigne, pistol)
    action.execute()

    print("\n--- TEST 6: ATTACK AGAIN ---")
    action = Attack(game, gascoigne, pistol)
    action.execute()

    print("\n--- TEST 7: USE HEALING ITEM ---")
    game.player.set_property("health", 70)  # Damage player first
    print(f"Current health: {game.player.get_property('health')}")
    action = Heal(game)
    action.execute()

    print("\n--- TEST 8: INSPECT WEAPON ---")
    action = Inspect(game, saw_cleaver)
    action.execute()

    print("\n--- TEST 9: DROP AND GET ITEM ---")
    action = Drop(game, pistol)
    action.execute()
    print(f"Inventory: {', '.join(game.player.inventory.keys())}")

    pistol_on_ground = game.player.location.items["Hunter Pistol"]
    action = Get(game, pistol_on_ground)
    action.execute()
    print(f"Inventory: {', '.join(game.player.inventory.keys())}")

    print("\n" + "="*60)
    print("FINAL STATUS")
    print("="*60)
    print(f"Location: {game.player.location.name}")
    print(f"Health: {game.player.get_property('health')}/{game.player.get_property('max_health')}")
    print(f"Blood Echoes: {game.player.get_property('blood_echoes')}")
    print(f"Insight: {game.player.get_property('insight')}")
    print(f"Inventory: {', '.join(game.player.inventory.keys())}")
    print(f"Saw Cleaver Level: +{saw_cleaver.get_property('upgrade_level')}")
    print(f"Saw Cleaver Damage: {saw_cleaver.get_property('damage')}")
    print(f"Father Gascoigne Health: {gascoigne.get_property('health')}")
    print(f"Father Gascoigne Status: {'Dead' if gascoigne.get_property('is_dead') else 'Alive'}")

BLOODBORNE TEXT ADVENTURE - ACTION SYSTEM DEMO

Location: Hunter's Dream
Health: 100/100
Blood Echoes: 500
Insight: 0
Inventory: Saw Cleaver, Hunter Pistol, Blood Vial


--- TEST 1: UPGRADE WEAPON ---
✓ Upgraded Saw Cleaver from 25 to +1. Damage: 30

--- TEST 2: TRAVEL ---
✓ Teleported to Central Yharnam.

A decrepit street filled with the stench of beasts.


--- TEST 2B: TRAVEL BY WALKING ---
✗ You haven't lit the lamp at that location yet.
✗ You haven't lit the lamp at that location yet.
Player is now at: Central Yharnam

--- TEST 3: INTERACT WITH LAMP ---
✓ You lit the lamp. You can now travel here from other lamps.

--- TEST 4: TALK TO NPC ---
✓ Father Gascoigne: "Beasts all over the shop... You'll be one of them, sooner or later."

--- TEST 5: ATTACK ---
✓ You attack Father Gascoigne with the Hunter Pistol for 15 damage.

--- TEST 6: ATTACK AGAIN ---
✓ You attack Father Gascoigne with the Hunter Pistol for 15 damage.

--- TEST 7: USE HEALING ITEM ---
Current health: 70
✓ Healed fo